In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
max_words = 20000
max_len = 200
num_classes = 1

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [3]:
DATA_PATH = 'отзывы за лето.xls'

In [4]:
data = pd.read_excel(DATA_PATH)

In [5]:
data.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [6]:
data['target'] = (data['Rating'] >= 3) * 1
df_train = data[['Content', 'target']]

In [7]:
df_train.head()

,Content,target
0,It just works!,1
1,В целом удобноное приложение...из минусов хотя...,1
2,Отлично все,1
3,Стал зависать на 1% работы антивируса. Дальше ...,1
4,"Очень удобно, работает быстро.",1


## Предобработка

In [8]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [9]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [10]:
pd.set_option('mode.chained_assignment', None)
df_train['Content'] = df_train['Content'].apply(preprocess_text)

In [11]:
df_train.head()

,Content,target
0,it just works,1
1,целое удобноной приложениеиз минус хотеть боль...,1
2,отлично,1
3,зависать 1 работа антивирус ранее пользоваться...,1
4,удобно работать быстро,1


In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_val, y_train, y_val = train_test_split(df_train.drop(columns='target', inplace=False), df_train['target'], 
                                                  train_size=0.7, shuffle=True, random_state=27)

X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, train_size=0.7, shuffle=True, random_state=27)

In [14]:
train_corpus = " ".join(X_train['Content'])
train_corpus = train_corpus.lower()

In [15]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /Users/ian/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [17]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [18]:
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'отличный',
 'хороший',
 'телефон',
 'супер']

In [19]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [20]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [21]:
X_train = np.asarray([text_to_sequence(text, max_len) for text in X_train['Content']], dtype=np.int32)
X_test = np.asarray([text_to_sequence(text, max_len) for text in X_test['Content']], dtype=np.int32)
X_val = np.asarray([text_to_sequence(text, max_len) for text in X_val['Content']], dtype=np.int32)

In [22]:
X_train[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0], dtype=int32)

## Keras model

In [23]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, Reshape
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [24]:
num_classes = 2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [25]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 10))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [26]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['AUC'])

In [27]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss', patience=5)  


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_val, y_val),
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
 1/29 [>.............................] - ETA: 0s - loss: 0.7032 - auc: 0.2045WARNING:tensorflow:From /opt/anaconda3/envs/gb_nlp/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
29/29 [==============================] - 12s 423ms/step - loss: 0.4154 - auc: 0.8869 - val_loss: 0.3266 - val_auc: 0.9528
Epoch 2/20
29/29 [==============================] - 12s 420ms/step - loss: 0.2614 - auc: 0.9615 - val_loss: 0.2268 - val_auc: 0.9688
Epoch 3/20
29/29 [==============================] - 12s 413ms/step - loss: 0.1729 - auc: 0.9823 - val_loss: 0.2097 - val_auc: 0.9737
Epoch 4/20
29/29 [==============================] - 12s 415ms/step - loss: 0.1249 - auc: 0.9907 - val_loss: 0.2213 - val_auc: 0.9732
Epoch 5/20
29/29 [==============================] - 12s 416ms/step - loss: 0.0894 - auc: 0.9952 -

In [28]:
score = model.evaluate(X_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Val loss:', score[0])
print('Val AUC:', score[1])

9/9 [==============================] - 1s 75ms/step - loss: 0.3461 - auc: 0.9605


Val loss: 0.34613510966300964
Val AUC: 0.9604792594909668


In [29]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test AUC:', score[1])

4/4 [==============================] - 0s 58ms/step - loss: 0.3624 - auc: 0.9577


Test loss: 0.36237868666648865
Test AUC: 0.9577283263206482


## Word2vec

In [30]:
import gensim.models

In [31]:
df_train['tokens'] = df_train['Content'].map(nltk.tokenize.word_tokenize)

In [32]:
t = Tokenizer(num_words=1000)
t.fit_on_texts(df_train['Content'].values)
vocab_size = len(t.word_index) + 1
vocab_size

13514

In [35]:
w2v_model = gensim.models.Word2Vec(sentences=df_train['tokens'], size=128, window=5, 
                                   sg=1, hs=0, negative=10, workers=32, seed=34)

In [36]:
w2v_model.train(sentences=train_corpus, total_examples=w2v_model.corpus_count, epochs=20)

(687520, 12175480)

In [37]:
embedding_matrix = np.zeros((vocab_size, 128))
for word, i in t.word_index.items():
    try:
        embedding_vector = w2v_model[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        pass

/opt/anaconda3/envs/gb_nlp/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [38]:
a = [i for i in range(len(embedding_matrix)) if any(embedding_matrix[i])]
len(a)

2105

In [39]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len, trainable=False, 
                    weights=[embedding_matrix]))
model.add(Conv1D(128, 10))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [40]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['AUC'])

In [41]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_val, y_val),
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
29/29 [==============================] - 9s 313ms/step - loss: 0.4011 - auc: 0.9059 - val_loss: 0.3264 - val_auc: 0.9348
Epoch 2/20
29/29 [==============================] - 9s 302ms/step - loss: 0.3044 - auc: 0.9430 - val_loss: 0.3022 - val_auc: 0.9432
Epoch 3/20
29/29 [==============================] - 9s 306ms/step - loss: 0.2871 - auc: 0.9498 - val_loss: 0.2913 - val_auc: 0.9466
Epoch 4/20
29/29 [==============================] - 9s 310ms/step - loss: 0.2739 - auc: 0.9547 - val_loss: 0.2857 - val_auc: 0.9489
Epoch 5/20
29/29 [==============================] - 9s 313ms/step - loss: 0.2620 - auc: 0.9590 - val_loss: 0.2830 - val_auc: 0.9504
Epoch 6/20
29/29 [==============================] - 9s 312ms/step - loss: 0.2546 - auc: 0.9611 - val_loss: 0.2799 - val_auc: 0.9514
Epoch 7/20
29/29 [==============================] - 9s 313ms/step - loss: 0.2478 - auc: 0.9630 - val_loss: 0.2736 - val_auc: 0.9538
Epoch 8/20
29/29 [==============================] - 9s 314ms/step - loss: 0.

In [42]:
score = model.evaluate(X_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Val loss:', score[0])
print('Val AUC:', score[1])

9/9 [==============================] - 1s 66ms/step - loss: 0.2824 - auc: 0.9524


Val loss: 0.28241947293281555
Val AUC: 0.9523863792419434


In [43]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test AUC:', score[1])

4/4 [==============================] - 0s 52ms/step - loss: 0.2799 - auc: 0.9527


Test loss: 0.27988284826278687
Test AUC: 0.9527285099029541


У сети на претренированных эмбеддингах, тренировка шла более гладко, меньше заметно переобучение, хотя финальная точность немного ниже. Затрудняюсь точно сказать, с чем связано, пока не успел до конца вникнуть в тему сверточных сетей для задач NLP